# Weather Chatbot

This notebook creates a weather chatbot with whom the user can interactively ask weather-related questions.  
The implementation requires the model to interact with a weather lookup tool.

In [ ]:
from dotenv import load_dotenv
from openai import OpenAI
from datetime import datetime
import gradio as gr
import urllib.parse
import requests
import json

In [ ]:
LLAMA_MODEL = "llama3.2"
ollama = OpenAI(base_url='http://localhost:11434/v1', api_key='dummy')

In [ ]:
cur_date = datetime.today().strftime('%Y-%m-%d')
system_prompt = "You are a weather chatbot that specializes in answering questions about current weather and weather forecasts for global cities. "
system_prompt += "You can only provide current weather, or forecasts for the past 3 days, current day, or coming 3 days. "
system_prompt += "If the user asks for weather outside these parameters, politely apologize and state the restrictions. "
system_prompt += f"Today's date is {cur_date}. "

In [ ]:
def geo_lookup(city):
    encoded = urllib.parse.quote_plus(city)
    response = requests.get(f"https://geocoding-api.open-meteo.com/v1/search?name={encoded}&count=1&language=en&format=json")
    
    if response.status_code == 200:
        json_data = response.json()
        if 'results' in json_data:
            lat = json_data['results'][0]['latitude']
            lon = json_data['results'][0]['longitude']
            return lat, lon
        else:
            print(f"No result found for {city}.")
            return None, None
    else:
        print(f"Request failed with status code: {response.status_code}")
        return None, None

In [ ]:
def get_forecast(city):
    lat, lon = geo_lookup(city)
    if lat:
        response = requests.get(f"https://api.open-meteo.com/v1/forecast?latitude={lat}&longitude={lon}&current=temperature_2m,relative_humidity_2m,precipitation,rain,showers,snowfall,weather_code,wind_speed_10m,wind_direction_10m&daily=weather_code,temperature_2m_max,temperature_2m_min,precipitation_sum,rain_sum,showers_sum,snowfall_sum,precipitation_hours,precipitation_probability_max,wind_speed_10m_max,wind_gusts_10m_max,wind_direction_10m_dominant&past_days=3&forecast_days=4")
        
        if response.status_code == 200:
            json_data = response.json()
            print(json_data)
            return json_data
        else:
            print(f"Request failed with status code: {response.status_code}")
    else:
        print(f"City {city} not found.")

In [ ]:
get_forecast("Toronto")

In [ ]:
description = "Get the current weather and past or future weather forecast for a given city name. "
description += "Call this whenever you need to retrieve weather data, for example when a user asks "
description += "'What's the current weather in Tokyo?' or 'What is tomorrow's forecast for Toronto?' or 'What was yesterday's weather in Paris?'. "
description += "You can only retrieve current weather, or forecasts for the past 3 days, current day, or coming 3 days. "
description += "The relevant weather data can be extracted from the returned JSON data as follows. "
description += "Current weather data is in the 'current' field. "
description += "Past, present and future forecast data is in the 'daily' field. "
description += "Within the 'daily' field, look up the requested date in the 'time' array, then find the forecast details in the other arrays at the corresponding index. "
description += "Translate weather codes as follows: "
description += "0-Clear sky, 1-Mainly clear, 2-Partly cloudy, 3-Overcast, 45-Fog, 48-Depositing rime fog, 51-Light drizzle, 53-Moderate drizzle, 55-Dense drizzle, "
description += "56-Light freezing drizzle, 57-Dense freezing drizzle, 61-Slight rain, 63-Moderate rain, 65-Heavy rain, 66-Light freezing rain, 67-Heavy freezing rain, "
description += "71-Slight snow fall, 73-Moderate snow fall, 75-Heavy snow fall, 77-Snow grains, 80-Slight rain showers, 81-Moderate rain showers, "
description += "82-Violent rain showers, 85-Slight snow showers, 86-Heavy snow showers, 95-Thunderstorm, 96-Thunderstorm with slight hail, 99-Thunderstorm with heavy hail. "

forecast_fn = {
    "name": "get_forecast",
    "description": description,
    "parameters": {
        "type": "object",
        "properties": {
            "city": {
                "type": "string",
                "description": "The name of the city for which to retrieve the forecast",
            },
        },
        "required": ["city"],
        "additionalProperties": False
    }
}

In [ ]:
tools = [
    {"type": "function", "function": forecast_fn},
]

In [ ]:
def chat(message, history):
    messages = [{"role": "system", "content": system_prompt}] + history + [{"role": "user", "content": message}]
    response = ollama.chat.completions.create(model=LLAMA_MODEL, messages=messages, tools=tools)

    while response.choices[0].finish_reason=="tool_calls":
        message = response.choices[0].message
        response = handle_tool_call(message)
        messages.append(message)
        messages.append(response)
        response = ollama.chat.completions.create(model=LLAMA_MODEL, messages=messages, tools=tools)
    
    return response.choices[0].message.content

In [ ]:
def handle_tool_call(message):
    tool_call = message.tool_calls[0]
    arguments = json.loads(tool_call.function.arguments)
    city = arguments.get('city')
    weather = get_forecast(city)
    response = {
        "role": "tool",
        "content": json.dumps(weather),
        "tool_call_id": tool_call.id
    }
    return response

In [ ]:
gr.ChatInterface(fn=chat, type="messages").launch(inbrowser=True)